## Working page leafthrough

In [ ]:
import gradio as gr

def paginate(page, max_page, page_change):
    new_page = page + page_change
    new_page = max(1, new_page)  # Ensures page is not less than 1
    new_page = min(new_page, max_page)  # Ensures page does not exceed max_page
    return new_page

def paginate_go(page, max_page):
    try:
        page = int(page)
    except ValueError:
        print(f'Invalid page number: {page}')
        return None
    return paginate(page, max_page, 0)
def create_gradio_app():
    with gr.Blocks() as app:
        with gr.Row():
            page_input = gr.Number(label="Page Number", value=1)
            max_page = gr.Number(label="Max Page", value=10)  # Assuming max 10 pages for testing
            go_button = gr.Button("Go to Page")
            next_button = gr.Button("Next")
            prev_button = gr.Button("Previous")
            current_page_label = gr.Label("Current Page: 1")

        def update_page_label(page):
            current_page_label.update(f"Current Page: {page}")

        go_button.click(fn=paginate_go, inputs=[page_input, max_page], outputs=page_input)
        next_button.click(fn=lambda x, y: paginate(x, y, 1), inputs=[page_input, max_page], outputs=page_input)
        prev_button.click(fn=lambda x, y: paginate(x, y, -1), inputs=[page_input, max_page], outputs=page_input)
        page_input.change(update_page_label)

    return app

app = create_gradio_app()
app.launch()

## Working going through images across different pages

In [ ]:
import gradio as gr
import math
import os

IMAGES_TO_SHOW = 1  # Set to 1 since we're showing one audio file per page
loaded_audios = []  # Global variable to store the loaded audios

def paginate(page, max_page, page_change):
    new_page = page + page_change
    new_page = max(1, new_page)
    new_page = min(new_page, max_page)
    return new_page

def get_audio_for_page(all_audios, page):
    if 1 <= page <= len(all_audios):
        return all_audios[page - 1]
    return None  # Return None if the page is out of range

def update_audio_display(page):
    global loaded_audios
    max_page = math.ceil(len(loaded_audios) / IMAGES_TO_SHOW)
    displayed_audio = get_audio_for_page(loaded_audios, page)
    audio_name = os.path.basename(displayed_audio) if displayed_audio else ""
    current_page_label_text = f"Current Page: {page}/{max_page}"
    return displayed_audio, audio_name, page, current_page_label_text

def handle_pagination(page, delta):
    global loaded_audios
    max_page = math.ceil(len(loaded_audios) / IMAGES_TO_SHOW)
    new_page = paginate(page, max_page, delta)
    return update_audio_display(new_page)

def handle_audio_load(audios):
    global loaded_audios
    loaded_audios = audios
    return update_audio_display(1)  # Display first audio

def create_gradio_app():
    with gr.Blocks() as app:
        with gr.Row():
            audio_loader = gr.File(label="Load Audio Files", file_count='multiple')
            audio_player = gr.Audio(label="Audio Player")
            audio_name_box = gr.Textbox(label="Audio File Name", interactive=True)

        with gr.Row():
            page_input = gr.Number(label="Page Number", value=1, visible=True)
            current_page_label = gr.Label("Current Page: 1/X")
            go_button = gr.Button("Go to Page")
            prev_button = gr.Button("Previous")
            next_button = gr.Button("Next")

        audio_loader.change(fn=handle_audio_load, inputs=[audio_loader], outputs=[audio_player, audio_name_box, page_input, current_page_label])
        next_button.click(fn=lambda page: handle_pagination(page, 1), inputs=[page_input], outputs=[audio_player, audio_name_box, page_input, current_page_label])
        prev_button.click(fn=lambda page: handle_pagination(page, -1), inputs=[page_input], outputs=[audio_player, audio_name_box, page_input, current_page_label])
        go_button.click(fn=lambda page: update_audio_display(page), inputs=[page_input], outputs=[audio_player, audio_name_box, page_input, current_page_label])

    return app

app = create_gradio_app()
app.launch()


## Display audio files per page

In [ ]:
import gradio as gr
import math

def paginate(page, max_page, page_change):
    new_page = page + page_change
    new_page = max(1, new_page)
    new_page = min(new_page, max_page)
    return new_page

def get_audio_for_page(all_audios, page):
    if 1 <= page <= len(all_audios):
        return all_audios[page - 1]
    return None  # Return None if the page is out of range

def update_audio_display(page, max_page):
    global loaded_audios
    displayed_audio = get_audio_for_page(loaded_audios, page)
    return displayed_audio, page, f"Current Page: {page}"

def handle_pagination(page, max_page, delta):
    new_page = paginate(page, max_page, delta)
    return update_audio_display(new_page, max_page)

def handle_audio_load(audios):
    global loaded_audios
    loaded_audios = audios
    max_page = len(loaded_audios)
    return update_audio_display(1, max_page)  # Display first audio

def create_gradio_app():
    global loaded_audios
    loaded_audios = []

    with gr.Blocks() as app:
        with gr.Row():
            audio_loader = gr.File(label="Load Audio Files", file_count='multiple')
            audio_player = gr.Audio(label="Audio Player")

        with gr.Row():
            page_input = gr.Number(label="Page Number", value=1, visible=True)
            current_page_label = gr.Label("Current Page: 1")
            go_button = gr.Button("Go to Page")
            next_button = gr.Button("Next")
            prev_button = gr.Button("Previous")

        audio_loader.change(fn=handle_audio_load, inputs=[audio_loader], outputs=[audio_player, page_input, current_page_label])
        next_button.click(fn=lambda page: handle_pagination(page, len(loaded_audios), 1), inputs=[page_input], outputs=[audio_player, page_input, current_page_label])
        prev_button.click(fn=lambda page: handle_pagination(page, len(loaded_audios), -1), inputs=[page_input], outputs=[audio_player, page_input, current_page_label])
        go_button.click(fn=lambda page: update_audio_display(page, len(loaded_audios)), inputs=[page_input], outputs=[audio_player, page_input, current_page_label])

    return app

app = create_gradio_app()
app.launch()
